<a href="https://colab.research.google.com/github/bibhs0401/TextSummarization/blob/main/Cosine_similarity_for_text_summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import re
import nltk
import string
import numpy as np
import networkx as nx
from nltk.cluster.util import cosine_distance

In [3]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
stopwords = nltk.corpus.stopwords.words('english')
print(stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [5]:
def preprocess(text):
  formatted_text = text.lower()
  tokens = []
  for token in nltk.word_tokenize(formatted_text):
    tokens.append(token)
  #print(tokens)
  tokens = [word for word in tokens if word not in stopwords and word not in string.punctuation]
  formatted_text = " ".join(element for element in tokens)
  return formatted_text

In [6]:
original_text = """Machine learning (ML) is an umbrella term for solving problems for which
development of algorithms by human programmers would be cost-prohibitive, and instead the problems are solved by helping
machines discover their own algorithms, without needing to be explicitly told what to do by any human-developed algorithms.
Recently, generative artificial neural networks have been able to surpass results of many previous approaches.
Machine-learning approaches have been applied to large language models, computer vision, speech recognition, email filtering, agriculture and medicine,
where it is too costly to develop algorithms to perform the needed tasks.
The mathematical foundations of ML are provided by mathematical optimization (mathematical programming) methods.
Data mining is a related (parallel) field of study, focusing on exploratory data analysis through unsupervised learning.
ML is known in its application across business problems under the name predictive analytics.
Although not all machine learning is statistically based, computational statistics is an important source of the fields methods"""
original_text = re.sub(r'\s'," ", original_text)
original_text

'Machine learning (ML) is an umbrella term for solving problems for which development of algorithms by human programmers would be cost-prohibitive, and instead the problems are solved by helping machines discover their own algorithms, without needing to be explicitly told what to do by any human-developed algorithms. Recently, generative artificial neural networks have been able to surpass results of many previous approaches. Machine-learning approaches have been applied to large language models, computer vision, speech recognition, email filtering, agriculture and medicine, where it is too costly to develop algorithms to perform the needed tasks. The mathematical foundations of ML are provided by mathematical optimization (mathematical programming) methods. Data mining is a related (parallel) field of study, focusing on exploratory data analysis through unsupervised learning. ML is known in its application across business problems under the name predictive analytics. Although not all 

**Function to calculate the similarity between the sentences**

In [7]:
original_sentences = [sentence for sentence in nltk.sent_tokenize(original_text)]
original_sentences

['Machine learning (ML) is an umbrella term for solving problems for which development of algorithms by human programmers would be cost-prohibitive, and instead the problems are solved by helping machines discover their own algorithms, without needing to be explicitly told what to do by any human-developed algorithms.',
 'Recently, generative artificial neural networks have been able to surpass results of many previous approaches.',
 'Machine-learning approaches have been applied to large language models, computer vision, speech recognition, email filtering, agriculture and medicine, where it is too costly to develop algorithms to perform the needed tasks.',
 'The mathematical foundations of ML are provided by mathematical optimization (mathematical programming) methods.',
 'Data mining is a related (parallel) field of study, focusing on exploratory data analysis through unsupervised learning.',
 'ML is known in its application across business problems under the name predictive analyti

In [8]:
formatted_sentences = [preprocess(original_sentence) for original_sentence in original_sentences]
formatted_sentences

['machine learning ml umbrella term solving problems development algorithms human programmers would cost-prohibitive instead problems solved helping machines discover algorithms without needing explicitly told human-developed algorithms',
 'recently generative artificial neural networks able surpass results many previous approaches',
 'machine-learning approaches applied large language models computer vision speech recognition email filtering agriculture medicine costly develop algorithms perform needed tasks',
 'mathematical foundations ml provided mathematical optimization mathematical programming methods',
 'data mining related parallel field study focusing exploratory data analysis unsupervised learning',
 'ml known application across business problems name predictive analytics',
 'although machine learning statistically based computational statistics important source fields methods']

In [27]:
def calculate_sentence_similarity(sentence1, sentence2):
  words1 = [word for word in nltk.word_tokenize(sentence1)]
  words2 = [word for word in nltk.word_tokenize(sentence2)]
  #print(words1)
  #print(words2)
  all_words = list(set(words1+words2))
  #print(all_words)
  vector1 = [0]* len(all_words)
  vector2 = [0]* len(all_words)
  #print(vector1)

  for word in words1:
    #print(word)
    vector1[all_words.index(word)] +=1
  for word in words2:
    #print(word)
    vector2[all_words.index(word)] +=1
  #print(vector2)
  return 1-cosine_distance(vector1, vector2)


vector:value for words appearing once will be 1 and so on

In [30]:
calculate_sentence_similarity(formatted_sentences[0],formatted_sentences[0])

1.0

**Function to create similarity matrix**
The higher the value, the greater the similarity between the sentences.
The more words in common, the greater the similarity

In [36]:
def calculate_similarity_matrix(sentences):
  similarity_matrix = np.zeros((len(sentences), len(sentences)))
  #print(similarity_matrix)
  for i in range(len(sentences)):
    for j in range(len(sentences)):
      if i == j:
        continue
      similarity_matrix[i][j] = calculate_sentence_similarity(sentences[i], sentences[j])
  return similarity_matrix

In [37]:
calculate_similarity_matrix(formatted_sentences)

array([[0.        , 0.        , 0.11504475, 0.04428074, 0.04583492,
        0.17149859, 0.10341754],
       [0.        , 0.        , 0.06741999, 0.        , 0.        ,
        0.        , 0.        ],
       [0.11504475, 0.06741999, 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.04428074, 0.        , 0.        , 0.        , 0.        ,
        0.0860663 , 0.07784989],
       [0.04583492, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.0805823 ],
       [0.17149859, 0.        , 0.        , 0.0860663 , 0.        ,
        0.        , 0.        ],
       [0.10341754, 0.        , 0.        , 0.07784989, 0.0805823 ,
        0.        , 0.        ]])

Function to summarize the texts

In [60]:
def summarize(text, num_of_sentences, percentage=0):
  original_sentences = [sentence for sentence in nltk.sent_tokenize(text)]
  formatted_sentences = [preprocess(original_sentence) for original_sentence in original_sentences]
  similarity_matrix = calculate_similarity_matrix(formatted_sentences)

  similarity_graph = nx.from_numpy_array(similarity_matrix)
  #print(similarity_graph.nodes)
  #print(similarity_graph.edges)

  scores = nx.pagerank(similarity_graph)
  #print(scores)
  ordered_scores = sorted(((scores[i], score) for i, score in enumerate(original_sentences)), reverse = True)
  #print(ordered_scores)

  if percentage >0:
    num_of_sentences = int(len(formatted_sentences)*percentage)

  best_sentences = []
  for sentence in range(num_of_sentences):
    best_sentences.append(ordered_scores[sentence][1])

  return original_sentences, best_sentences, ordered_scores


In [45]:
summarize(original_text, 3)
#nodes give the position
#edges give which node is related to which node

[0, 1, 2, 3, 4, 5, 6]
[(0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (1, 2), (3, 5), (3, 6), (4, 6)]
{0: 0.2809707313331177, 1: 0.06293471007282339, 2: 0.13215400661245982, 3: 0.12681393575768507, 4: 0.08602923713712321, 5: 0.15130377999183206, 6: 0.15979359909495877}
[(0.2809707313331177, 'Machine learning (ML) is an umbrella term for solving problems for which development of algorithms by human programmers would be cost-prohibitive, and instead the problems are solved by helping machines discover their own algorithms, without needing to be explicitly told what to do by any human-developed algorithms.'), (0.15979359909495877, 'Although not all machine learning is statistically based, computational statistics is an important source of the fields methods'), (0.15130377999183206, 'ML is known in its application across business problems under the name predictive analytics.'), (0.13215400661245982, 'Machine-learning approaches have been applied to large language models, computer vision, speech r

In [47]:
original_sentences, best_sentences, scores = summarize(original_text, 3)

In [48]:
original_sentences

['Machine learning (ML) is an umbrella term for solving problems for which development of algorithms by human programmers would be cost-prohibitive, and instead the problems are solved by helping machines discover their own algorithms, without needing to be explicitly told what to do by any human-developed algorithms.',
 'Recently, generative artificial neural networks have been able to surpass results of many previous approaches.',
 'Machine-learning approaches have been applied to large language models, computer vision, speech recognition, email filtering, agriculture and medicine, where it is too costly to develop algorithms to perform the needed tasks.',
 'The mathematical foundations of ML are provided by mathematical optimization (mathematical programming) methods.',
 'Data mining is a related (parallel) field of study, focusing on exploratory data analysis through unsupervised learning.',
 'ML is known in its application across business problems under the name predictive analyti

In [49]:
best_sentences

['Machine learning (ML) is an umbrella term for solving problems for which development of algorithms by human programmers would be cost-prohibitive, and instead the problems are solved by helping machines discover their own algorithms, without needing to be explicitly told what to do by any human-developed algorithms.',
 'Although not all machine learning is statistically based, computational statistics is an important source of the fields methods',
 'ML is known in its application across business problems under the name predictive analytics.']

In [50]:
scores

[(0.2809707313331177,
  'Machine learning (ML) is an umbrella term for solving problems for which development of algorithms by human programmers would be cost-prohibitive, and instead the problems are solved by helping machines discover their own algorithms, without needing to be explicitly told what to do by any human-developed algorithms.'),
 (0.15979359909495877,
  'Although not all machine learning is statistically based, computational statistics is an important source of the fields methods'),
 (0.15130377999183206,
  'ML is known in its application across business problems under the name predictive analytics.'),
 (0.13215400661245982,
  'Machine-learning approaches have been applied to large language models, computer vision, speech recognition, email filtering, agriculture and medicine, where it is too costly to develop algorithms to perform the needed tasks.'),
 (0.12681393575768507,
  'The mathematical foundations of ML are provided by mathematical optimization (mathematical pro

In [51]:
def visualize(title, sentence_list, best_sentences):
  from IPython.core.display import HTML
  text = ""
  display(HTML(f'<h1>Summary - {title}</h1>'))
  for sentence in sentence_list:
    #print(sentence)
    #text += sentence
    if sentence in best_sentences:
      text += " "+sentence.replace(sentence, f"<mark>{sentence}</mark>")
    else:
      text += " "+sentence
  display(HTML(f"""{text}"""))

In [53]:
visualize('ML', original_sentences, best_sentences)

In [54]:
!pip install goose3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.7/88.7 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 17.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 10.4 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993224 sha256=80e31d5f0e8a9e2d12ee49df944bec77fd23fb67ff3563c3e025cf9925333a8c
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [56]:
g = Goose()
url = 'https://en.wikipedia.org/wiki/Automatic_summarization'
article = g.extract(url)

In [63]:
original_sentences, best_sentences, scores = summarize(article.cleaned_text, 120, 0.5)

In [64]:
visualize(article.title, original_sentences, best_sentences)